In [31]:
"""
CryptoPanic API Wrapper.
Docs for API found here. https://cryptopanic.com/developers/api/
CryptoPanic Server is cached every 30 seconds. So crawling at a faster rate is pointless.
All API methods are rate limited per IP at 5req/sec.
"""
#import datetime
from Deepl.api_deepl import translate_news
import requests
import json
import pandas as pd
from config import API_KEY
import time

global_api_rate_delay = 0.2  # All API methods are rate limited per IP at 5req/sec.

class CryptoPanic():

    url = f'https://cryptopanic.com/api/v1/posts/?auth_token={API_KEY}&public=true'

    def __init__(self, url = url, currencies = None, kind = None, region = None, page = None):
        """
        Handles URL variables for API POST
        """

        if currencies is not None:
            if len(currencies.split(',')) <= 50:
                self.currencies = currencies
                url += "&currencies={}".format(currencies)
            else:
                print("Warning: Maximum number of currencies is 50")

        kinds = ['news', 'media']
        if kind is not None and kind in kinds:
            self.kind = kind
            url += "&kind={}".format(kind)
        elif kind is not None and kind not in kinds:
            print('Warning: Kind is not available')

        regions = ['en', 'de', 'es', 'fr', 'it', 'pt', 'ru']  # (English), (Deutsch), (Español), (Français), (Italiano), (Português), (Русский)--> Respectively
        if region is not None and region in regions:
            self.region = region
            url += "&region={}".format(region)
        elif region is not None and region not in regions:
            print('Warning: Region is not available')

        if page is not None:
            url += f"&page={page}"
        if region == 'fr':
            self.url = 'https://cryptopanic.com/api/v1/posts/?auth_token=f3322769569c9c952301819406f554c1c818cf6d&public=true&kind=news&regions=fr'
        else:
            self.url = url

    def get_whole_page(self):
        page = requests.get(self.url)
        my_json = json.loads(page.text)
        return my_json

    def get_news_titles(self, number):
        """
        Get news titles
        Returns: List of news titles
        """
        time.sleep(global_api_rate_delay)
        news_titles = []
        my_json_page = self.get_whole_page()

        for i in range(number):
            news_titles.append(my_json_page["results"][i]["title"])

        return news_titles

    def get_source(self, number):
        """
        Gets news sources
        Returns: List of news' sources
        """
        time.sleep(global_api_rate_delay)
        news_sources = []
        my_json_page = self.get_whole_page()

        for i in range(number):
            news_sources.append(my_json_page["results"][i]["domain"])

        return news_sources

    def aggregate_data(self, number):
        """
        Gathers sources and news titles 
        Returns: Dictionary
        """
        if self.region == 'en':
            translated_news = self.get_news_titles(number) #translate_news(self.get_news_titles(self.number))
        else:
            translated_news = self.get_news_titles(number)
        sources = self.get_source(number)
        dict = {}
        j = 0
        for i in range(len(translated_news)):
            dict[translated_news[i]] = sources[j]
            j += 1
        return dict


    def get_df(self):
        """
        Returns: pandas DF
        """
        news = self.get_news_titles(self.number) 
        translated_news = translate_news(self.get_news_titles(self.number))
        sources = self.get_source(self.number)
        df = pd.DataFrame(news, columns =['News'])
        df['Translated News'] = translated_news
        df['Sources'] = sources

        return df
    
    def get_last_news(self, number):
            return self.get_news_titles(number)[0]

In [34]:
test1 = CryptoPanic(kind = 'news', region = 'en')
test2 = CryptoPanic(kind = 'news', region = 'fr')
test1.get_news_titles(3)
test2.get_news_titles(3)
def main():
    LOOP1 = True
    last_en_news = test1.get_last_news(3)
    last_french_news = test2.get_last_news(3)
    print(last_en_news)
    print(last_french_news)
    while LOOP1:
        boolean1 = False
        boolean2 = False
        while not boolean1 or not boolean2:
            new_last_en_news = test1.get_last_news(3)
            new_last_french_news = test2.get_last_news(3)
            if last_en_news != new_last_en_news:
                print(new_last_en_news)
                last_en_news = new_last_en_news
                boolean1 = True
            if last_french_news != new_last_french_news:
                print(new_last_french_news)
                last_french_news = new_last_french_news
                boolean2 = True  
        LOOP1 = False    
main()

Lithium Finance designated oracle for pricing synthetic private assets on Beyond Finance
Gene Simmons, le légendaire bassiste du groupe Kiss, est « all in » sur le Bitcoin
L’ogre du Bitcoin Coinbase révèle des résultats trimestriels hors-norme
Le géant de l’informatique Accenture visé par un ransomware Bitcoin (BTC)
Soccer star Messi's latest contract reportedly includes crypto fan tokens


In [25]:
test = CryptoPanic(kind = 'news', region = 'fr')
whole_page = test.get_whole_page()
news = test.get_news_titles(3)
sources = test.get_source(3)
print(news)
print(sources)
final = test.aggregate_data(3)
print(final)
print(test.get_last_news(3))

['Un analyste voit le Bitcoin atteindre 200 000 dollars en 2021 s’il parvient à franchir des niveaux clés', 'Cardano distribue des NFT un peu particuliers – Le cours de l’ADA en hausse', 'En plus du PSG, Lionel Messi débarque dans les NFT et envoie la cryptomonnaie Ethernity (ERN) en pleine lucarne']
['crypto-france.com', 'cryptoast.fr', 'journalducoin.com']
{'Un analyste voit le Bitcoin atteindre 200 000 dollars en 2021 s’il parvient à franchir des niveaux clés': 'crypto-france.com', 'Cardano distribue des NFT un peu particuliers – Le cours de l’ADA en hausse': 'cryptoast.fr', 'En plus du PSG, Lionel Messi débarque dans les NFT et envoie la cryptomonnaie Ethernity (ERN) en pleine lucarne': 'journalducoin.com'}
[]


In [15]:
test1 = CryptoPanic(kind = 'news', region = 'en')
test1.get_df()

,News,Translated News,Sources
0,DEXs could see demand boost as regulators targ...,Les DEX pourraient voir leur demande augmenter...,cointelegraph.com
1,"ETH Crosses $3,000 as Crypto Bill Goes to the ...","L'ETH franchit la barre des 3 000 $, alors que...",trustnodes.com
2,"Ethereum Rises Above $3,000 for First Time Sin...",L'Ethereum passe au-dessus de 3 000 $ pour la ...,decrypt.co
3,"Ethereum (ETH) Recaptures $3,000 After “London...",L'Ethereum (ETH) reprend 3 000 $ après le déma...,u.today
4,Are DAOs a new way for companies to go public?,Les DAO sont-ils un nouveau moyen pour les ent...,forkast.news
5,Crypto Whales Shift $4.8 Billion in Bitcoin as...,"Les baleines cryptographiques déplacent 4,8 mi...",u.today
6,Crypto Market Cap Taps a 2-Month Record as Bit...,La capitalisation boursière des crypto-monnaie...,cryptopotato.com
7,Elon Musk Says It’s Wrong Time for U.S. Senate...,Elon Musk dit que ce n'est pas le moment pour ...,u.today
8,What form of digital assets will be the future...,Quelle forme d'actifs numériques sera l'avenir...,cointelegraph.com
9,"Bitcoin Whales Suddenly Move $1,700,000,000 in...",Les baleines du bitcoin déplacent soudainement...,dailyhodl.com


In [2]:
test2 = CryptoPanic(kind = 'news', region = 'fr')
test2.get_df().News.iloc[0]

'JPMorgan offre à ses clients fortunés la possibilité de miser sur 6 crypto-fonds'

In [3]:
test1 = CryptoPanic(kind = 'news', region = 'en')
test2 = CryptoPanic(kind = 'news', region = 'fr')

In [4]:
df1 = test1.get_df()
df2 = test2.get_df()
last_en_news = test1.get_df().News.iloc[0]
last_french_news = test2.get_df().News.iloc[0]

In [5]:
last_en_news

"Ethereum price soars above $3K into 'red zone' triggering sell-off fears"

In [7]:
print(last_french_news)

JPMorgan offre à ses clients fortunés la possibilité de miser sur 6 crypto-fonds


In [ ]:
test1 = CryptoPanic(kind = 'news', region = 'en')
test2 = CryptoPanic(kind = 'news', region = 'fr')
LOOP = True
while LOOP:
    test1.get_df().News.iloc[0]
    test2.get_df().News.iloc[0]